# Using _egobox_ optimizer _Egor_

## Installation

In [13]:
%pip install egobox

Note: you may need to restart the kernel to use updated packages.


We import _egobox_ as _egx_ for short.

In [14]:
import numpy as np
import egobox as egx

You may setup the logging level to get optimization progress during the execution

In [15]:
# To display optimization information (none by default)
# import logging
# logging.basicConfig(level=logging.INFO)

## Example 1 : Continuous optimization

### Test functions

In [16]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [17]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


### Continuous optimization with _Egor_

In [18]:
egor = egx.Egor(g24, xspecs_g24, 
                     n_doe=10, 
                     n_cstr=n_cstr_g24, 
                     cstr_tol=[1e-3, 1e-3],
                     infill_strategy=egx.InfillStrategy.WB2,
                     target=-5.5,
                     # outdir="./out",
                     # hot_start=True
                    )  # see help(egor) for options

# Specify regression and/or correlation models used to build the surrogates of objective and constraints
#egor = egx.Egor(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [19]:
res = egor.minimize(n_iter=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-5.50823898e+00 -1.02637525e-03  7.31165162e-04] at [2.32965679 3.1785822 ]
Optimization history: 
Inputs = [[1.00714219 2.83680177]
 [2.01726227 0.97229664]
 [0.64443743 1.49235589]
 [2.98520626 2.47781049]
 [0.51144494 3.69603895]
 [1.39143602 0.0521766 ]
 [2.19062703 2.07848306]
 [1.79749186 0.5511925 ]
 [0.04433832 3.41661282]
 [2.42351384 1.66930727]
 [2.31595596 3.18731429]
 [2.32965679 3.1785822 ]]
Outputs = [[-3.84394396e+00 -1.16299419e+00  2.83599142e+00]
 [-2.98955891e+00 -1.03012858e+00 -3.02531983e+00]
 [-2.13679332e+00 -2.03391158e+00 -1.31360327e+00]
 [-5.46301675e+00 -1.68216678e+01  2.47436043e+00]
 [-4.20748389e+00  5.36841086e-01 -2.21660239e+00]
 [-1.44361262e+00 -3.38188883e+00 -1.53365927e+00]
 [-4.26911010e+00 -2.70284316e-01 -1.63608961e+00]
 [-2.34868436e+00 -1.71380930e+00 -3.12745825e+00]
 [-3.46095114e+00  1.40157532e+00 -2.84971328e+01]
 [-4.09282111e+00 -2.43765241e+00 -1.02446667e+00]
 [-5.50327026e+00  1.16427163e-01 -5.39230013e-02]
 [-5

## Example 2 : Mixed-integer optimization

### Test function

In [20]:
xspecs_mixint_xsinx = [egx.XSpec(egx.XType(egx.XType.INT), [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

### Mixed-integer optimization with _Egor_

In [21]:
egor = egx.Egor(mixint_xsinx, xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(n_iter=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-15.12161154] at [19.]
Optimization history: 
Inputs = [[10.]
 [ 7.]
 [22.]
 [25.]
 [21.]
 [20.]
 [19.]]
Outputs = [[  5.70983099]
 [  3.14127616]
 [ -7.10960014]
 [ 11.42919546]
 [-11.44370682]
 [-14.15453288]
 [-15.12161154]]


## Example 3 : More mixed-integer optimization

In the following example we see we can have other special integer type cases, where a component of x can take one value out of a list of ordered values (ORD type) or being like an enum value (ENUM type). Those types differ by the processing related to the continuous relaxation made behind the scene:
* For INT type, resulting float is rounded to the closest int value,
* For ORD type, resulting float is cast to closest value among the given valid ones,
* For ENUM type, one hot encoding is performed to give the resulting value.  

### Test function

In [22]:
# Objective function which takes [FLOAT, ENUM1, ENUM2, ORD] as input
# Note that ENUM values are passed as indice value eg either 0, 1 or 2 for a 3-sized enum  
def mixobj(X):
    # float
    x1 = X[:, 0]
    #  ENUM 1
    c1 = X[:, 1]
    x2 = c1 == 0
    x3 = c1 == 1
    x4 = c1 == 2
    #  ENUM 2
    c2 = X[:, 2]
    x5 = c2 == 0
    x6 = c2 == 1
    # int
    i = X[:, 3]

    y = (x2 + 2 * x3 + 3 * x4) * x5 * x1 + (x2 + 2 * x3 + 3 * x4) * x6 * 0.95 * x1 + i
    return y.reshape(-1, 1)

### Mixed-integer optimization with _Egor_

In [23]:
xtypes = [
    egx.XSpec(egx.XType(egx.XType.FLOAT), [-5.0, 5.0]),
    egx.XSpec(egx.XType(egx.XType.ENUM), tags=["blue", "red", "green"]),
    egx.XSpec(egx.XType(egx.XType.ENUM), xlimits=[2]),
    egx.XSpec(egx.XType(egx.XType.ORD), [0, 2, 3]),
]
egor = egx.Egor(mixobj, xtypes, seed=42)
res = egor.minimize(n_iter=10)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-14.25] at [-5.  2.  1.  0.]
Optimization history: 
Inputs = [[-1.90197486  2.          1.          3.        ]
 [ 1.36933896  1.          0.          2.        ]
 [-0.10843099  1.          0.          0.        ]
 [-4.73477511  0.          0.          3.        ]
 [ 3.11266243  2.          1.          2.        ]
 [ 0.33069418  2.          1.          0.        ]
 [ 4.47594664  2.          1.          0.        ]
 [-3.26619512  0.          0.          2.        ]
 [-5.          2.          1.          2.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          1.          0.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]]
Outp

Note that `x_opt` result contains indices for corresponding optional tags list hence the second component should be read as 0="red", 1="green", 2="blue", while the third component was unamed 0 correspond to first enum value and 1 to the second one.

## Usage

In [24]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(fun, n_cstr=0, n_start=20, n_doe=0, regression_spec=7, correlation_spec=15, infill_strategy=1, q_points=1, par_infill_strategy=1, infill_optimizer=1, n_clusters=1)
 |  
 |  Optimizer constructor
 |  
 |  fun: array[n, nx]) -> array[n, ny]
 |       the function to be minimized
 |       fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |          obj is the objective function [n, nx] -> [n, 1]
 |          cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |          an k the number of constraints (n_cstr)
 |          hence ny = 1 (obj) + k (cstrs)
 |       cstr functions are expected be negative (<=0) at the optimum.
 |  
 |   n_cstr (int):
 |       the number of constraint functions.
 |  
 |   cstr_tol (list(n_cstr)):
 |       List of tolerances for constraints to be satisfied (cstr < tol), its size shoulb equalt to n_cstr.
 |       None by default meaning zero tolerances.
 |  
 |   xspecs (list(XSpec)) where XS